### Compassus data preprocessing

Imports and read the data from provided csv.

In [6]:
#import libs
import sweetviz as sv
import pandas as pd
from pathlib import Path  
import sqlite3

In [2]:
data=pd.read_csv("GeorgiaTech-Data-Revised-v5.csv")

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (16,17,18,22,108,109,110,111,112,116,117,118,120,129,203) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [9]:
data['ceas_ascode'] = pd.to_numeric(data['ceas_ascode'], errors='coerce')

In [4]:
sel_cols = list(data.select_dtypes(include='object'))
data[sel_cols] = data[sel_cols].fillna('').astype(str)

In [5]:
advert_report = sv.analyze(data,pairwise_analysis='off')

In [7]:
advert_report.show_html('CompassusData.html')

Report CompassusData.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


28122

In [14]:

no_of_rows=len(data)
for col in data:
    if (data[col].isna().sum() / no_of_rows)*100 == 100.0:
        data.drop(col, inplace=True, axis = 1)

data.columns.size

152

In [15]:
filepath = Path('data_filtered.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
data.to_csv(filepath)  

In [16]:
def getColumnDtypes(dataTypes):
    dataList = []
    for x in dataTypes:
        if(x == 'int64'):
            dataList.append('int')
        elif (x == 'float64'):
            dataList.append('float')
        elif (x == 'bool'):
            dataList.append('boolean')
        else:
            dataList.append('varchar')
    return dataList

In [17]:
columnName = list(data.columns.values)
columnDataType = getColumnDtypes(data.dtypes)

In [18]:
createTableStatement = 'CREATE TABLE IF NOT EXISTS data ('
for i in range(len(columnDataType)):
    createTableStatement = createTableStatement + '\n' + columnName[i] + ' ' + columnDataType[i] + ','
createTableStatement = createTableStatement[:-1] + ' );'

In [19]:
database=Path('compassus_data.db')
if database.is_file():
    database.unlink()
database.touch()
conn = sqlite3.connect('compassus_data.db')
c = conn.cursor()
c.execute(createTableStatement)

data.to_sql('data', conn, if_exists='append', index = False)